# Physics of Behavior Tutorial II

In this session, we will focus on how to uncover long timescale properties of complex dynamical systems through a transfer operator approach.

Before we start, load the following python packages:

In [2]:
# Environment setup
!pip install -q -r https://raw.githubusercontent.com/AntonioCCosta/DPG_PhysicsBehavior_2026/main/requirements.txt

# Clone course repo (optional, for datasets)
!git clone https://github.com/AntonioCCosta/DPG_PhysicsBehavior_2026.git

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

fatal: destination path 'DPG_PhysicsBehavior_2026' already exists and is not an empty directory.
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.113+-x86_64-with-glibc2.35


In [ ]:
import